In [1]:
%matplotlib inline

In [2]:
from functools import partial

import numpy as np
from qdev_wrappers.dataset.doNd import do0d, do1d, do2d


from qcodes.dataset.database import initialise_database
from qcodes.dataset.experiment_container import new_experiment
from qcodes.dataset.plotting import plot_by_id
import plottr
import qcodes.config

# Live plotting

This should perfrom live plotting automatically provided that you have plottr installed and running from https://github.com/kouwenhovenlab/plottr




In [3]:
initialise_database()  # just in case no database file exists
new_experiment("doNd-tutorial", sample_name="no sample")

doNd-tutorial#no sample#110@c:\Users\jenielse\mymainfolder\experiments.db
-------------------------------------------------------------------------

In [4]:
# preparatory mocking of physical setup
from qcodes.tests.instrument_mocks import DummyInstrument
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dmm = DummyInstrument('dmm', gates=['v1', 'v2'])

In [5]:
# and we'll make a 2D gaussian to sample from/measure
def gauss_model(x0: float, y0: float, sigma: float, noise: float=0.0005):
    """
    Returns a generator sampling a gaussian. The gaussian is
    normalised such that its maximal value is simply 1
    """
    while True:
        (x, y) = yield
        model = np.exp(-((x0-x)**2+(y0-y)**2)/2/sigma**2)*np.exp(2*sigma**2)
        noise = np.random.randn()*noise
        yield model + noise

In [6]:
# and finally wire up the dmm v1 to "measure" the gaussian

gauss = gauss_model(0.1, 0.2, 0.25)
next(gauss)

def measure_gauss(dac):
    val = gauss.send((dac.ch1.get(), dac.ch2.get()))
    next(gauss)
    return val

dmm.v1.get = partial(measure_gauss, dac)
dmm.v2.get = np.random.rand

In [7]:
dataid, ax, cbax = do1d(dac.ch1, 0, 1, 100, 0.1, dmm.v1, dmm.v2)

Starting experimental run with id: 906
plot by id took 0.09566903114318848


In [10]:
dataid, ax, cbax = do2d(dac.ch1, 0, 1, 100, 0, 
                        dac.ch2, 0, 0.5, 100, 0, 
                        dmm.v1, dmm.v2,
                        do_plot=True)

Starting experimental run with id: 908
plot by id took 0.31467628479003906


# Adding some actions

Below we will demonstarte that it's possible to execute actions both before and after each stage on the loops. We will leave these actions as simple prints so that one can see when they are triggered.

In [8]:
def before_action():
    print("called before the loop")

def after_action():
    print("called after the loop")
    
def at_each_action():
    print("called at each step")
    
def before_inner_action():
    print("called before inner loop")
    
def after_inner_action():
    print("called after inner loop")

In [9]:
dataid = do1d(dac.ch1, 0, 1, 10, 0.01, dmm.v1, dmm.v2, at_each_action,
              enter_actions=(before_action,), exit_actions=(after_action,))

called before the loop
Starting experimental run with id: 907
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called at each step
called after the loop
plot by id took 0.06827116012573242


In [11]:
dataid = do2d(dac.ch1, 0, 1, 3, 0.01, 
              dac.ch2, 0, 0.5, 2, 0.01, 
              dmm.v1, dmm.v2, at_each_action,
              before_inner_actions=(before_inner_action,),
              after_inner_actions=(after_inner_action,),
              enter_actions=(before_action,), exit_actions=(after_action,))

called before the loop
Starting experimental run with id: 909
called before inner loop
called at each step
called at each step
called after inner loop
called before inner loop
called at each step
called at each step
called after inner loop
called before inner loop
called at each step
called at each step
called after inner loop
called after the loop
plot by id took 0.12787485122680664
